In [ ]:
import time, json
from datetime import datetime, timezone

def utc_now():
    return datetime.now(timezone.utc).isoformat()

try: runId = runId
except NameError: runId = f"manual_{int(time.time())}"

try: workloadName = workloadName
except NameError: workloadName = "unknown"

try: tier = tier
except NameError: tier = "unknown"

try: weight = weight
except NameError: weight = "0"

In [ ]:
import time, json
from pyspark.sql import functions as F
from pyspark.sql.window import Window

start_ts = time.time()
start_utc = utc_now()

# Medium dataset (~8M). Tune up/down by changing n.
n = 8_000_000

# Fact-like table
fact = (spark.range(0, n)
            .withColumn("customer_id", (F.col("id") % 500_000).cast("int"))
            .withColumn("amount", (F.rand(seed=11) * 1000).cast("double"))
            .withColumn("day", (F.col("id") % 30).cast("int"))
            .repartition(120, "customer_id"))  # moderate shuffle pressure

# Dimension-like table (smaller)
dim = (spark.range(0, 500_000)
           .withColumnRenamed("id", "customer_id")
           .withColumn("segment", (F.col("customer_id") % 20).cast("int")))

# Join + window aggregation (typical “reporting”)
joined = fact.join(F.broadcast(dim), on="customer_id", how="inner")

w = Window.partitionBy("segment").orderBy(F.desc("amount"))
top = (joined.withColumn("rn", F.row_number().over(w))
             .where(F.col("rn") <= 50)
             .groupBy("segment")
             .agg(F.count("*").alias("top_count"),
                  F.avg("amount").alias("avg_amount")))

rows = top.count()  # force execution

duration = round(time.time() - start_ts, 2)
result = {
    "runId": runId, "workloadName": workloadName, "tier": tier, "weight": weight,
    "startTimeUtc": start_utc, "durationSec": duration,
    "metric": {"resultRows": rows, "n": n}
}
print(result)



In [ ]:
mssparkutils.notebook.exit(json.dumps(result))